In [1]:
import sqlite3 as db
import datetime as dt

In [2]:
import numpy as np
import pandas as pd

%precision 3

pd.set_option('display.max_rows', 50) # max. rows to show
pd.set_option('display.max_columns', 15) # max. cols to show
pd.options.display.float_format = '{:,.3f}'.format # float precisions

In [5]:
finfolio = db.connect(r"D:\pOrgz-dev\finfolio-db-original.db") # connect to master db, finalized

In [9]:
transactions = pd.read_sql("SELECT * FROM 'ams.transactions'", finfolio)
transactions.sample(3)

,_id,AccountID,trxDate,trxType,trxAmount,trxDescOrig,_trxDescDrv,_trxMethod,_trxAccount,_self_account_transfer,_tag_by_automation,_tag_by_end_user,created_on,updated_on
1006,1007,9BC11EF0-16F3-4DE4-A6DB-01AF8775D7E6,2017-11-06,WITHDRAW,70.000,by debit card-OTHPOS143459 THE YELLOW STRAW ...,None,None,None,0,None,None,2022-01-01 10:00:00,None
1104,1105,9BC11EF0-16F3-4DE4-A6DB-01AF8775D7E6,2018-06-25,DEPOSIT,50.000,CREDIT INTEREST-- ref:: nan,None,INTEREST,None,0,None,None,2022-01-01 11:00:00,2024-03-10 17:10:05
2596,2598,9BC11EF0-16F3-4DE4-A6DB-01AF8775D7E6,2021-10-29,DEPOSIT,"1,857.000",BY TRANSFER- UPI/CR/130213761006/SOHA\nM S/SB...,None,UPI,None,0,None,None,2024-04-01 05:00:00,2024-03-10 16:50:35


In [11]:
transactions["trxAmount"] = transactions["trxAmount"].round(2)

In [14]:
# transactions.to_sql("ams.transactions", finfolio, index = False, if_exists = "append")

4840

In [13]:
transactions.isnull().sum()

_id                          0
AccountID                    0
trxDate                      0
trxType                      0
trxAmount                    0
trxDescOrig                  0
_trxDescDrv               4840
_trxMethod                1006
_trxAccount               4584
_self_account_transfer       0
_tag_by_automation        4840
_tag_by_end_user          4815
created_on                   0
updated_on                 972
dtype: int64

## Transaction Method: `NEFT`

In [38]:
def _trxMethod(description : str, tags : dict) -> list:
    method, account = None, None
    
    for _method in ["NEFT", "RTGS", "UPI", "MANDATE DEBIT"]:
        if _method in description:
            method = _method
            
    for _account, _account_name in tags.items():
        if _account in description:
            account = _account_name
    
    updated_on = "2024-03-10 16:50:35" if any([method, account]) else None
    return method, account, updated_on

In [39]:
tags = {
    "JASNI A S" : "JASNI A SALIM",
    "DIGITAL INDIAN" : "DIGITAL INDIAN PVT LTD",
    "RELIANCE JIO INF" : "RELIANCE JIO INFOCOMM LTD",
    "RELIANCE DIGITAL" : "RELIANCE JIO INFOCOMM LTD",
    "JIO PLATFORMS LI" : "RELIANCE JIO INFOCOMM LTD",
    "INSPIRIGENCE WOR" : "INSPIRIGENCE WORKS PVT LTD",
    "PIDILITE INDUSTRIES" : "PIDILITE INDUSTRIES PVT LTD",
    
    "ZERODHA" : "ZERODHA",
    "IIFL SECURITIES" : "IIFL SECURITIES"
}

In [40]:
transactions["_trxMethod"], transactions["_trxAccount"], transactions["updated_on"] = zip(*transactions["trxDescOrig"].apply(lambda x : _trxMethod(x, tags = tags)))

In [16]:
transactions["_trxMethod"].value_counts()

UPI         3320
NEFT         224
ATMCASH      136
MANDATE       62
INTEREST      60
IMPS          32
Name: _trxMethod, dtype: int64

In [8]:
transactions.fillna("").groupby(["_trxAccount", "_trxMethod", "trxType"])["trxAmount"].sum().reset_index()

,_trxAccount,_trxMethod,trxType,trxAmount
0,,,DEPOSIT,"3,633,060.250"
1,,,WITHDRAW,"5,889,802.620"
2,,ATMCASH,DEPOSIT,"2,500.000"
3,,ATMCASH,WITHDRAW,"347,500.000"
4,,IMPS,DEPOSIT,"167,155.880"
5,,IMPS,WITHDRAW,"88,233.180"
6,,INTEREST,DEPOSIT,"33,603.000"
7,,MANDATE,WITHDRAW,59.000
8,,NEFT,DEPOSIT,"405,918.310"
9,,NEFT,WITHDRAW,"45,200.000"
